In [8]:
# Importar las bibliotecas necesarias
import sys
import os
import pandas as pd
from sklearn.exceptions import NotFittedError
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#Configuracion de seaborn
sns.set_theme(style='whitegrid', context='paper', palette='muted')

# Agregar el directorio de scripts al path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'scripts'))

# Importar el logger personalizado
from logger import CustomLogger

# Inicializar el logger
logger = CustomLogger(developer='David')
app_logger = logger.get_logger('app')
errors_logger = logger.get_logger('errors')

try:
    # Cargar los datos de entrenamiento
    train_data = pd.read_csv('../data/train.csv')
    app_logger.info("Conjunto de datos de entrenamiento cargado exitosamente.")
except FileNotFoundError:
    errors_logger.error("No se pudo encontrar el archivo de datos de entrenamiento.")
    raise
except Exception as e:
    errors_logger.error(f"Error al cargar los datos de entrenamiento: {str(e)}")
    raise

# Cargamos test.csv
try:
    test_data = pd.read_csv('../data/test.csv')
    app_logger.info("Conjunto de datos de test cargado exitosamente.")
except FileNotFoundError:
    errors_logger.error("No se pudo encontrar el archivo de datos de test.")
    raise

# Mostrar las primeras filas del conjunto de datos
train_data.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [2]:
# Valores faltantes
print("Valores faltantes por columna:")
valores_faltantes = train_data.isnull().sum()
porcentaje_faltantes = 100 * train_data.isnull().sum() / len(train_data)
tabla_faltantes = pd.concat([valores_faltantes, porcentaje_faltantes], axis=1, keys=['Total', 'Porcentaje'])
print(tabla_faltantes[tabla_faltantes['Total'] > 0].sort_values('Total', ascending=False))

# Filas duplicadas
filas_duplicadas = train_data.duplicated().sum()
print(f"\nNúmero de filas duplicadas: {filas_duplicadas}")

# Registrar en el log
if valores_faltantes.sum() > 0:
    app_logger.info(f"Se encontraron {valores_faltantes.sum()} valores faltantes en total.")
else:
    app_logger.info("No se encontraron valores faltantes en el conjunto de datos.")

if filas_duplicadas > 0:
    app_logger.warning(f"Se encontraron {filas_duplicadas} filas duplicadas en el conjunto de datos.")
else:
    app_logger.info("No se encontraron filas duplicadas en el conjunto de datos.")


Valores faltantes por columna:
              Total  Porcentaje
PoolQC         1453   99.520548
MiscFeature    1406   96.301370
Alley          1369   93.767123
Fence          1179   80.753425
FireplaceQu     690   47.260274
LotFrontage     259   17.739726
GarageType       81    5.547945
GarageYrBlt      81    5.547945
GarageFinish     81    5.547945
GarageQual       81    5.547945
GarageCond       81    5.547945
BsmtExposure     38    2.602740
BsmtFinType2     38    2.602740
BsmtFinType1     37    2.534247
BsmtCond         37    2.534247
BsmtQual         37    2.534247
MasVnrArea        8    0.547945
MasVnrType        8    0.547945
Electrical        1    0.068493

Número de filas duplicadas: 0


In [9]:
# Valores faltantes en el conjunto de test
print("Valores faltantes por columna en el conjunto de test:")
valores_faltantes_test = test_data.isnull().sum()
porcentaje_faltantes_test = 100 * test_data.isnull().sum() / len(test_data)
tabla_faltantes_test = pd.concat([valores_faltantes_test, porcentaje_faltantes_test], axis=1, keys=['Total', 'Porcentaje'])
print(tabla_faltantes_test[tabla_faltantes_test['Total'] > 0].sort_values('Total', ascending=False))

# Filas duplicadas en el conjunto de test
filas_duplicadas_test = test_data.duplicated().sum()
print(f"\nNúmero de filas duplicadas en el conjunto de test: {filas_duplicadas_test}")

# Registrar en el log para el conjunto de test
if valores_faltantes_test.sum() > 0:
    app_logger.info(f"Se encontraron {valores_faltantes_test.sum()} valores faltantes en total en el conjunto de test.")
else:
    app_logger.info("No se encontraron valores faltantes en el conjunto de test.")

if filas_duplicadas_test > 0:
    app_logger.warning(f"Se encontraron {filas_duplicadas_test} filas duplicadas en el conjunto de test.")
else:
    app_logger.info("No se encontraron filas duplicadas en el conjunto de test.")


Valores faltantes por columna en el conjunto de test:
              Total  Porcentaje
PoolQC         1456   99.794380
MiscFeature    1408   96.504455
Alley          1352   92.666210
Fence          1169   80.123372
FireplaceQu     730   50.034270
LotFrontage     227   15.558602
GarageCond       78    5.346127
GarageYrBlt      78    5.346127
GarageQual       78    5.346127
GarageFinish     78    5.346127
GarageType       76    5.209047
BsmtCond         45    3.084304
BsmtExposure     44    3.015764
BsmtQual         44    3.015764
BsmtFinType1     42    2.878684
BsmtFinType2     42    2.878684
MasVnrType       16    1.096642
MasVnrArea       15    1.028101
MSZoning          4    0.274160
BsmtFullBath      2    0.137080
BsmtHalfBath      2    0.137080
Functional        2    0.137080
Utilities         2    0.137080
GarageCars        1    0.068540
GarageArea        1    0.068540
TotalBsmtSF       1    0.068540
KitchenQual       1    0.068540
BsmtUnfSF         1    0.068540
BsmtFinSF2        

## Missing Columns on Train
- Pool Quality Col has 99,52% missing values, but we still have Pool Area wich i think is more important than PoolQc
- MiscFeature Col has 96.30% missing values. We also have MiscVal that represents the value of Misc Feature.
    - Each Misc Feature has a unique value or has multiple values?? If each MiscFeature has a unique value we can drop MiscFeature and leave Misc Val because we wont lose info, otherwise, drop both.
- Alley has 93% missing values, no other col is related to this one so we can directly drop it.

## Missing Columns on Test
- We can observe the same tendency


## Verificar la relación unívoca entre MiscFeature y MiscVal

In [3]:
# Crear un DataFrame con MiscFeature y MiscVal
misc_df = train_data[['MiscFeature', 'MiscVal']]

# Agrupar por MiscFeature y contar los valores únicos de MiscVal
relacion_misc = misc_df.groupby('MiscFeature')['MiscVal'].nunique().reset_index()
relacion_misc.columns = ['MiscFeature', 'Valores_Unicos_MiscVal']

print("Relación entre MiscFeature y MiscVal:")
print(relacion_misc)

# Verificar si cada MiscFeature tiene un único valor de MiscVal
es_univoca = (relacion_misc['Valores_Unicos_MiscVal'] == 1).all()

if es_univoca:
    print("\nLa relación entre MiscFeature y MiscVal es unívoca.")
    app_logger.info("La relación entre MiscFeature y MiscVal es unívoca. Se puede considerar dejar una columna.")
else:
    print("\nLa relación entre MiscFeature y MiscVal no es unívoca.")
    app_logger.info("La relación entre MiscFeature y MiscVal no es unívoca. Se recomienda eliminar ambas columnas.")

# Mostrar ejemplos de MiscFeature con múltiples valores de MiscVal
if not es_univoca:
    print("\nEjemplos de MiscFeature con múltiples valores de MiscVal:")
    ejemplos_multiples = relacion_misc[relacion_misc['Valores_Unicos_MiscVal'] > 1]
    for _, row in ejemplos_multiples.iterrows():
        feature = row['MiscFeature']
        valores = misc_df[misc_df['MiscFeature'] == feature]['MiscVal'].unique()
        print(f"MiscFeature: {feature}")
        print(f"Valores de MiscVal: {valores}\n")


Relación entre MiscFeature y MiscVal:
  MiscFeature  Valores_Unicos_MiscVal
0        Gar2                       2
1        Othr                       2
2        Shed                      18
3        TenC                       1

La relación entre MiscFeature y MiscVal no es unívoca.

Ejemplos de MiscFeature con múltiples valores de MiscVal:
MiscFeature: Gar2
Valores de MiscVal: [15500  8300]

MiscFeature: Othr
Valores de MiscVal: [3500    0]

MiscFeature: Shed
Valores de MiscVal: [ 700  350  500  400  480  450 1200  800 2000  600 1300   54  620  560
 1400    0 1150 2500]



In [4]:
# Filtrar columnas con más del 90% de valores faltantes
umbral_faltantes = 0.9
columnas_a_eliminar = tabla_faltantes[tabla_faltantes['Porcentaje'] > 90].index
train_data_filtrado = train_data.drop(columns=columnas_a_eliminar)

# Registrar en el log
if len(columnas_a_eliminar) > 0:
    app_logger.info(f"Se eliminaron {len(columnas_a_eliminar)} columnas con más del 90% de valores faltantes: {', '.join(columnas_a_eliminar)}")
    print(f"Columnas eliminadas: {', '.join(columnas_a_eliminar)}")
else:
    app_logger.info("No se encontraron columnas con más del 90% de valores faltantes.")
    print("No se encontraron columnas con más del 90% de valores faltantes.")

# Mostrar la forma del nuevo conjunto de datos
print(f"\nForma del conjunto de datos original: {train_data.shape}")
print(f"Forma del conjunto de datos filtrado: {train_data_filtrado.shape}")


Columnas eliminadas: Alley, PoolQC, MiscFeature

Forma del conjunto de datos original: (1460, 81)
Forma del conjunto de datos filtrado: (1460, 78)


In [10]:
# Importar sweetviz
import sweetviz as sv

# Crear un informe de análisis exploratorio de datos con sweetviz
informe = sv.analyze(train_data_filtrado)

# Mostrar el informe en el navegador
informe.show_html("informe_sweetviz.html")

print("Se ha generado un informe de análisis exploratorio de datos con sweetviz.")
print("Por favor, abra el archivo 'informe_sweetviz.html' en su navegador para ver el informe completo.")

# Registrar en el log
app_logger.info("Se ha generado un informe de análisis exploratorio de datos utilizando sweetviz.")


                                             |          | [  0%]   00:00 -> (? left)

Report informe_sweetviz.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
Se ha generado un informe de análisis exploratorio de datos con sweetviz.
Por favor, abra el archivo 'informe_sweetviz.html' en su navegador para ver el informe completo.


In [11]:
informe = sv.analyze(train_data_filtrado, target_feat="SalePrice")
informe.show_html("informe_sweetviz_target.html")

                                             |          | [  0%]   00:00 -> (? left)

Report informe_sweetviz_target.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [12]:
# Importar dtale
import dtale

# Crear una instancia de dtale con los datos filtrados
d = dtale.show(train_data_filtrado)

# Mostrar el enlace para acceder a la interfaz de dtale
print("Se ha generado un análisis interactivo con dtale.")
print(f"Por favor, acceda a la siguiente URL para explorar los datos: {d._url}")

# Registrar en el log
app_logger.info("Se ha generado un análisis interactivo utilizando dtale.")


2024-07-21 17:55:33,519 - INFO     - Se ha generado un análisis interactivo utilizando dtale.


Se ha generado un análisis interactivo con dtale.
Por favor, acceda a la siguiente URL para explorar los datos: http://PortatilDavid:40000


## Cleaning 
- We have seen some Low Varianze variables, some categoricals with a few categories where just one of them was present on the dataset and some numerical variables with really low variance (this can bias the model or not matter at all when using regularization)
- 